## Домашняя работа
ФИО: Смирнов Александр Сергеевич

Группа: РИМ-150950

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`

## Решение


In [11]:
from threading import Lock 
# импорт класса Lock из модуля threading

class ThreadSafeSingleton(type):
    _instances = {}
    _lock = Lock()

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            with cls._lock:
                if cls not in cls._instances:  
                # двойная проверка для безопасности
                    cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

- ThreadSafeSingleton — метакласс, который контролирует создание экземпляров своих классов.
- Атрибут _instances хранит уже созданные экземпляры.
- _lock — общий для всех классов Lock для синхронизации в многопоточной среде.
- В методе __call__ используется двойная проверка if cls not in cls._instances. сначала проверка без блокировки, чтобы повысить скорость доступа при повторных вызовах, затем с блокировкой для безопасного создания экземпляра.

In [12]:
class DatabasePool(int, metaclass = ThreadSafeSingleton):
    def __init__(self):
        self._connection = 'connected'
    def get_connection(self):
        return self._connection

### проверка задания 1

In [14]:
# Создание 3 экземпляров DatabasePool
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()

In [15]:
# Необходимо убедитеться, что это один и тот же объект
assert pool1 is pool2 is pool3

In [17]:
# Проверка, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()
pool2._connection = 'disconnected'
conn2 = pool2.get_connection()
assert conn1 != conn2 # проверка условия соединений

## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
1. При создании экземпляра класса счетчик должен увеличиваться
1. Добавьте метод get_count(), который возвращает количество созданных экземпляров

## Решение

In [40]:
class CountInstancesMeta(type):
    _counters = {}

    def __call__(cls, *args, **kwargs): 
        # Создаётся экземпляр
        instance = super().__call__(*args, **kwargs) 
        # Увеличивается счётчик для класса
        cls._counters[cls] = cls._counters.get(cls, 0) + 1
        return instance

    def get_count(cls): 
        # Возвращается количество созданных экземпляров класса cls
        return cls._counters.get(cls, 0)

- Метакласс CountInstancesMeta имеет атрибут _counters, который хранит словарь с подсчётом экземпляров по каждому классу.
- Метод __call__ вызывается при создании экземпляра, и тогда увеличивается счетчик для конкретного класса.
- Метод get_count() вызывается у класса и возвращает текущее количество созданных экземпляров именно этого класса.

In [43]:
class User(metaclass = CountInstancesMeta):
    def __init__(self, name):
        self.name = name

class Product(metaclass = CountInstancesMeta):
    def __init__(self, name):
        self.name = name

### проверка задания 2

In [44]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

## Решение

In [ ]:
class DescribeMeta(type):
    def __new__(cls, name, bases, attrs):
        def describe(self):
        # Метод describe() возвращает строку с именем класса
            return f"Это объект класса {self.__class__.__name__}"
        # Добавляется метод в атрибуты класса
        attrs['describe'] = describe
        # Создается и возвращается класс
        return super().__new__(cls, name, bases, attrs)

- Метакласс DescribeMeta переопределяет метод __new__, чтобы при создании любого класса автоматически добавить метод describe.
- Метод describe возвращает строку с именем текущего класса через self.__class__.__name__.
- Затем обычные классы указывают в объявлении metaclass = DescribeMeta, чтобы получить добавленный метод.

In [46]:
class Car(metaclass = DescribeMeta):
    def __init__(self, brand):
        self.brand = brand

class Book(metaclass = DescribeMeta):
    def __init__(self, title):
        self.title = title

### проверка задания 3

In [47]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)
## Решение

In [49]:
class SaveMeta(type):
    def __new__(cls, name, bases, attrs):
        if 'save' not in attrs:
            raise TypeError(f"Класс {name} должен реализовать метод save()")
        return super().__new__(cls, name, bases, attrs)

In [50]:
class User(metaclass=SaveMeta):
    # Класс с User с методом save()
    def __init__(self, name):
        self.name = name

    def save(self):
        print(f"Пользователь {self.name} сохранён")

- В метаклассе SaveMeta в методе __new__ происходит проверка метода save на отсутствие в атрибутах класса и, если метода save нет, то выводится ошибка TypeError.
- Класс User содержит метод save() для успешного создания и использования объекта.
- Попытка создать класс Message без метода save вызовет ошибку TypeError сразу при объявлении.

### проверка задания 4

In [51]:
# Проверка
user = User("Alice")
user.save()  # Должно работать

Пользователь Alice сохранён


In [53]:
# Этот код должен вызвать ошибку:
class Message(metaclass = SaveMeta):
    def __init__(self, text):
        self.text = text

TypeError: Класс Message должен реализовать метод save()